# 11.Convolutinal Neural Network

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 1024
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [6]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.math.divide_no_nan(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

print(train_dataset)

<MapDataset shapes: ((None, 32, 32, 3), (None, 10)), types: (tf.float32, tf.float32)>


In [8]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.math.divide_no_nan(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
print(test_dataset)

<MapDataset shapes: ((None, 32, 32, 3), (None, 10)), types: (tf.float32, tf.float32)>


In [12]:
class Cifar10Model(tf.keras.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tf.keras.Sequential([
            L.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same'
            ),
        ])

        self.conv_block2 = tf.keras.Sequential([
            L.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
            ),
        ])
        
        self.conv_block3 = tf.keras.Sequential([
            L.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
            ),
        ])
        
        self.flatten = L.Flatten()
        self.fc1 = L.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = L.Dropout(rate=0.8)
        self.fc2 = L.Dense(10)
        self.softmax = L.Softmax()

    def call(self, x, training=False):
        x = self.conv_block1(x, training=training)
        x = self.conv_block2(x, training=training)
        x = self.conv_block3(x, training=training)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x), training=training)
        x = self.fc2(x)
        return self.softmax(x)

In [13]:
model = Cifar10Model()

def loss_fn(y, y_pre):
    return tf.keras.losses.categorical_crossentropy(y, y_pre)

def accuracy(y, y_pre):
    return tf.keras.metrics.categorical_accuracy(y, y_pre)

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [14]:
for j in range(num_epochs):
    
    running_loss = 0
    running_acc = 0

    for i, (x_, y_) in enumerate(train_dataset):
        
        with tf.GradientTape() as tape:
            y_pre = model(x_, training=True)
            loss = loss_fn(y_, y_pre)
        acc = accuracy(y_, y_pre)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        running_loss += tf.reduce_mean(loss)
        running_acc += tf.reduce_mean(acc)
    
    print("-----epoch {} -----".format(j + 1))
    print("loss: ", running_loss.numpy()/(i + 1))
    print("acc: ", running_acc.numpy()/(i + 1)) 

-----epoch 1 -----
loss:  2.176459176199777
acc:  0.18480797203219668
-----epoch 2 -----
loss:  1.8556046777841997
acc:  0.3186650957380022
-----epoch 3 -----
loss:  1.7119883323202327
acc:  0.3803419385637556
-----epoch 4 -----
loss:  1.6115576296436542
acc:  0.4134356440330038
-----epoch 5 -----
loss:  1.5665786120356346
acc:  0.4334392158352599
-----epoch 6 -----
loss:  1.5082090728136959
acc:  0.45704892216896525
-----epoch 7 -----
loss:  1.461917176538584
acc:  0.4739344071368782
-----epoch 8 -----
loss:  1.4254875961615114
acc:  0.49176484711316165
-----epoch 9 -----
loss:  1.4117657408422353
acc:  0.4957914157789581
-----epoch 10 -----
loss:  1.3693771362304688
acc:  0.5115299614108338


In [15]:
test_accuracy = 0
for i, (x_, y_) in enumerate(test_dataset):
    y_pre = model(x_)
    test_accuracy += tf.reduce_mean(accuracy(y_, y_pre))
test_accuracy /= i + 1

print("test accuracy {:0.3f}".format(test_accuracy.numpy()))

test accuracy 0.556
